In [1]:
import psycopg2
import pandas as pd
import numpy as np
import gensim
import re
import warnings
from scipy import spatial
from nltk.corpus import stopwords
import warnings

In [7]:
class BookLens:
    def __init__(self, dbname, user, password, host, port):
        self.connection = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )

    def _preprocess(self, text):
        text = text.lower()
        text = re.sub(r"[^\w\s]", "", text)
        text = text.encode('utf-8', 'ignore').decode('utf-8')
        stop_words = stopwords.words("english")
        text = " ".join([word for word in text.split() if word not in stop_words])
        return text

    def _create_word2vec_model(self, df):
        cleaned_titles = df["title"].apply(self._preprocess)
        
        tokenized_titles = cleaned_titles.apply(lambda x: x.split())

        return gensim.models.Word2Vec(sentences=tokenized_titles, vector_size=100, window=15, min_count=1, sg=0)

    def create_and_save_word2vec_model(self, model_filename):
        df = pd.read_sql_query("SELECT * FROM books", self.connection)
        book_model = self._create_word2vec_model(df)
        book_model.save(model_filename)
        return book_model

    def analyze_books(self, user_query, model_filename):
        loaded_model = gensim.models.Word2Vec.load(model_filename)
        processed_query = self._preprocess(user_query)
        tokenized_query = processed_query.split()
        query_vector = np.mean([loaded_model.wv[word] if word in loaded_model.wv else np.zeros(100) for word in tokenized_query], axis=0)

        df = pd.read_sql_query("SELECT * FROM books", self.connection)

        book_scores = []
        for idx, book in enumerate(df["title"]):
            book_vector = np.mean([loaded_model.wv[word] if word in loaded_model.wv else np.zeros(100) for word in self._preprocess(book).split()], axis=0)
            similarity_score = 1 - spatial.distance.cosine(query_vector, book_vector)
            id, title, author, price, description, rating, num_of_rating = df.iloc[idx][["id", "title", "author", "price", "description", "rating", "num_of_rating"]]
            combined_score = similarity_score * (rating / 5)
            book_scores.append((id, title, author, combined_score, price, description, rating, num_of_rating))

        top_books = pd.DataFrame(book_scores, columns=["id", "title", "author", "score", "price", "description", "rating", "num_of_rating"]).nlargest(5, "score")

        top_books_ids = tuple(top_books["id"].tolist())
        query = f"""
            SELECT reviews.review_id,
                reviews.reviewer_name,
                reviews.review_rating,
                reviews.review_title,
                reviews.review_date,
                reviews.review_content,
                reviews.book_id,
                books.title
            FROM reviews
            JOIN books ON reviews.book_id = books.id
            WHERE reviews.book_id in {top_books_ids}"""
        reviews = pd.read_sql_query(query, self.connection)

        def title_similarity(book_title, review_content):
            book_title = self._preprocess(book_title)
            review_content = self._preprocess(review_content)
            book_title = book_title.split()
            review_content = review_content.split()
            book_vector = np.mean([loaded_model.wv[word] if word in loaded_model.wv else np.zeros(100) for word in book_title], axis=0)
            review_vector = np.mean([loaded_model.wv[word] if word in loaded_model.wv else np.zeros(100) for word in review_content], axis=0)
            similarity = 1 - spatial.distance.cosine(book_vector, review_vector)
            return similarity

        reviews["review_score"] = reviews.apply(lambda x: title_similarity(x["title"], x["review_title"] + " " + x["review_content"]), axis=1)
        top_reviews = pd.concat([reviews[reviews["book_id"] == book_id].nlargest(5, ["review_score", "review_rating"]) for book_id in top_books_ids]).drop_duplicates()

        return top_books, top_reviews

    def close_connection(self):
        self.connection.close()

In [13]:
# Filter out the specific warning
warnings.filterwarnings("ignore", message="pandas only support SQLAlchemy connectable")
# Filter out the specific runtime warning
warnings.filterwarnings("ignore", message="invalid value encountered in double_scalars")



# Example usage:
dbname = 'booklens'
user = '##############'
password = '#############'
host = 'localhost'
port = '5432'
model_filename = "book_model.bin"
user_query = "excel"

analyzer = BookLens(dbname, user, password, host, port)
book_model = analyzer.create_and_save_word2vec_model(model_filename)
top_books, top_reviews = analyzer.analyze_books(user_query, model_filename)
analyzer.close_connection()


In [10]:
top_books

,id,title,author,score,price,description,rating,num_of_rating
145,179,R for Excel Users: An Introduction to R for Ex...,John L Taveras,0.583234,18.00,"R has a steep learning curve and, if taken in ...",4.5,139
67,69,Hands-On Financial Modeling with Microsoft Exc...,Shmuel Oluwa,0.544748,33.49,Explore the aspects of financial modeling with...,4.2,114
150,184,Hands-On Financial Modeling with Microsoft Exc...,Shmuel Oluwa,0.544748,33.49,Explore the aspects of financial modeling with...,4.2,114
52,53,Data Analysis with Excel: Tips and tricks to k...,Manisha Nigam,0.540743,15.96,Simplify the Usage of Excel Features and Funct...,4.0,143
136,157,Excel: The Absolute Beginner's Guide to Maximi...,Henry Skinner,0.525835,18.97,Ready to Transform Your Excel Skills? 🌟 EXCLU...,4.4,146


In [11]:
top_reviews

,review_id,reviewer_name,review_rating,review_title,review_date,review_content,book_id,title,review_score
32,2313,Kyle & Heather,5.0,Excel to R Translator,2020-08-09,I'd consider myself a strong Excel user - I us...,179,R for Excel Users: An Introduction to R for Ex...,0.803009
31,2312,Matan Gilbert,4.0,Very good introduction to R for Excel users,2017-05-14,Very good introduction to R for Excel users. L...,179,R for Excel Users: An Introduction to R for Ex...,0.735860
38,2319,Amazon Customer,5.0,Excellent Introduction to R,2016-09-13,This book is an excellent introduction to R. I...,179,R for Excel Users: An Introduction to R for Ex...,0.665703
34,2315,J.,4.0,Great intro to R,2016-10-30,"Great intro to R, especially the sections that...",179,R for Excel Users: An Introduction to R for Ex...,0.647827
33,2314,Adriel Irons,5.0,For those who want to take it to the next level,2016-08-14,"I use Excel on a constant basis for work, but ...",179,R for Excel Users: An Introduction to R for Ex...,0.624101
16,933,HA3,1.0,Quite vague!,2020-11-09,"Quite vague, not what I was expecting for. Ver...",69,Hands-On Financial Modeling with Microsoft Exc...,1.000000
10,927,Stanley Tan,4.0,For beginners who are learning to setup Financ...,2020-07-20,Clear and simple instructions for you to follo...,69,Hands-On Financial Modeling with Microsoft Exc...,0.477764
11,928,Jane,5.0,Very good for beginners,2020-05-22,"This is a very good book for beginners, howeve...",69,Hands-On Financial Modeling with Microsoft Exc...,0.409747
12,929,Paul O'Sullivan,2.0,Not worth it,2020-09-16,I read the first 40 something pages and put th...,69,Hands-On Financial Modeling with Microsoft Exc...,0.340700
14,931,JCK,1.0,Crap product,2022-08-06,Book is nearly worthless. The practice tools a...,69,Hands-On Financial Modeling with Microsoft Exc...,0.164776
